To stop processes
```bash
pkill -9 -f pw.x
pkill -9 -f gipaw.x

```

In [ ]:
import f90nml
nml = f90nml.write('sample.nml')


In [ ]:
crystal=read("test/MIN-167-350K-CuCbPyz.cif")

In [ ]:
from ase.io import read, write
from ase.visualize import view
# crystal=read("../structures/KTU-183_2_auto.cif")
crystal_264=read("../structures/new_systems/ktu_002.cif")
crystal_258=read("../structures/new_systems/KTU-065-25do-b.cif")
crystal_612=read("../structures/new_systems/KTU-TBS-20do_auto.cif")
# crystal=read("../structures/KTU-183_2_auto_dimer.xyz")
view([crystal_264, crystal_258, crystal_612])

In [ ]:
from ase.visualize.plot import plot_atoms
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
plot_atoms(crystal_264, ax, radii=0.5, rotation=('0x,0y,90z'))
# fig.savefig("ase_slab.png")

In [ ]:
print(len(crystal1), len(crystal2), len(crystal3))

# Plot experimental spectra

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nmr_spectrum_264 = np.loadtxt("../structures/new_systems/spectra/20220318_13C_Mingoo-KTU-O-TIPS-re.txt", dtype=float)
nmr_spectrum_264_x = np.linspace(283.67840576171875, -68.08218583648267, num=8192)
nmr_spectrum_258 = np.loadtxt("../structures/new_systems/spectra/20211217_13C_Mingoo-KTU-TIPS-re.txt", dtype=float)
nmr_spectrum_258_x = np.linspace(283.9461975097656, -67.8144882603051, num=8192)
nmr_spectrum_612 = np.loadtxt("../structures/new_systems/spectra/20211202_13C_Mingoo-KTU-TBS.txt", dtype=float)
nmr_spectrum_612_x = np.linspace( 283.67840576171875, -67.8144882603051, num=8187)

print(nmr_spectrum_264.shape)

plt.plot(nmr_spectrum_264_x, nmr_spectrum_264)
plt.gca().invert_xaxis()
plt.xlabel("chemical shift, ppm")
plt.show()
plt.plot(nmr_spectrum_258_x, nmr_spectrum_258)
plt.gca().invert_xaxis()
plt.xlabel("chemical shift, ppm")
plt.show()
plt.plot(nmr_spectrum_612_x, nmr_spectrum_612)
plt.gca().invert_xaxis()
plt.xlabel("chemical shift, ppm")
plt.show()

In [ ]:
nmr_spectrum_612.shape

# Main calculation

In [ ]:
%%time
%load_ext autoreload
%autoreload 2

from runner import pw_runner
from ase.io import read, write

import time

import os

# Defining parameters:

np_pw = 13
np_gipaw = 13

pw_params = {
    'prefix':'crystal', 
    'restart_mode' : 'from_scratch',
    'tstress':True, 
    'tprnfor':True, 
    'nosym':True, 
    'ecutwfc':10, 
    'kpts':(1, 1, 1),
    # 'kpts':None, 
    'ecutrho' : 100,
    # 'occupations' : 'smearing', 
    # 'smearing' : 'gauss', 
    # 'degauss' : 1.0e-2
}

# Loading structures:

# crystal=read("../structures/MIN-167-350K-CuCbPyz.cif")
# crystal=read("../structures/HIK-143 293K-activated.cif")
# crystal=read("../structures/HIK-143 MeOH.cif")
# crystal=read("../structures/KTU-183_2_auto.cif")
crystal_264=read("../structures/new_systems/ktu_002.cif")
crystal_258=read("../structures/new_systems/KTU-065-25do-b.cif")
crystal_612=read("../structures/new_systems/KTU-TBS-20do_auto.cif")

crystal = crystal_258

tms = read("../structures/tms.xyz")
# crystal = tms
a = 10.0
tms.set_cell([a, a, a])
tms.set_cell([(a, 0, 0), (0, a, 0), (0, 0, a)])
tms.set_pbc(True)

# Creating directory to store all the data:
calc_dir = pw_runner.make_calc_dir()


print("Starting QE/SCF calculation for TMS...")
pw_runner.run_pw_scf(calc_dir, tms, num_proc_pw=np_pw, pw_params=pw_params)
print("Starting gipaw calculation for TMS...")
gipaw_out = pw_runner.run_gipaw(calc_dir, "espresso_gipaw_tms.pwo", num_proc_gipaw=np_gipaw)
chemical_shifts_iso, chemical_shifts_tensors = pw_runner.parse_gipaw_output(calc_dir + '/espresso_gipaw_tms.pwo', num_atoms=len(tms))
tms.info['chemical_shifts_iso'] = chemical_shifts_iso
tms.info['chemical_shifts_tensors'] = chemical_shifts_tensors
write(calc_dir+"/tms.xyz", tms, format='extxyz')
os.rename(calc_dir+"/espresso.pwo", calc_dir+"/espresso_tms.pwo")

print("Starting QE/SCF calculation for crystal...")
pw_runner.run_pw_scf(calc_dir, crystal, num_proc_pw=np_pw, pw_params=pw_params)
print("Starting gipaw calculation for crystal...")
gipaw_out = pw_runner.run_gipaw(calc_dir, "espresso_gipaw.pwo", num_proc_gipaw=np_gipaw)
chemical_shifts_iso, chemical_shifts_tensors = pw_runner.parse_gipaw_output(calc_dir + '/espresso_gipaw.pwo', num_atoms=len(crystal))
crystal.info['chemical_shifts_iso'] = chemical_shifts_iso
crystal.info['chemical_shifts_tensors'] = chemical_shifts_tensors
write(calc_dir+"/crystal.xyz", crystal, format='extxyz')

In [ ]:
from runner.plot import plot_chemical_shifts

path1 = "/media/dlb/la_cie1/repos/pw-benchmarks/qe/test/2023-01-10_17 40-57_007593"
path2 = "/media/dlb/la_cie1/repos/pw-benchmarks/qe/test/2023-01-11_18 05-03_816467"

crystal1 = read(path1+"/crystal.xyz", format="extxyz")
crystal2 = read(path2+"/crystal.xyz", format="extxyz")

ch1= plot_chemical_shifts(crystal1, "./test")
ch2 = plot_chemical_shifts(crystal2, "./test")

print(ch1['H']-ch2['H']) 
# print(ch1['C']-ch2['C'])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(1, len(scf_times)+1, 1, dtype=int)
plt.plot(x, scf_times_5, label='SCF time')
plt.plot(x, gipaw_times_5, label='GIPAW time')
plt.xticks(x)
plt.xlabel('Num. of MPI processes')
plt.ylabel('Time, s.')
plt.legend()
plt.show()

In [ ]:
x

In [ ]:
print(start_time, scf_time, gipaw_time-start_time)

In [ ]:
struct = read(calc_dir+"/crystal.xyz", format="extxyz")
print( struct.get_potential_energy() )
print( struct.get_forces() )
print( struct.info['chemical_shifts_iso'])
print( struct.info['chemical_shifts_tensors'])

In [ ]:
import psutil
 
# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

crystal=read("test/MIN-167-350K-CuCbPyz.cif")
crystal.get_forces()

In [ ]:
import numpy as np

from ase.build import molecule

import torch

atoms = molecule('CH3CH2NH2')

store_data = {
        'aaa': np.random.randn(300, 300),
        'bbb': "hello"
}


torch_data = torch.rand(2000, 3000)

atoms.info['store_data'] = store_data

atoms.info['torch_data'] = torch_data



%store atoms

In [ ]:
# $HOME/.ipython/profile_default/db/autorestore
%store


In [ ]:
%store 